En este notebook, aprendará cómo utilizar búsqueda léxica en combinación con búsqueda vectorial.

Author: Brian O'Grady

Date: 11/11/2024

Notebook adaptado de: [Vector Search Quickstart](https://docs.datastax.com/en/cql/dse/getting-started/vector-search-quickstart.html)

### Instalar dependencias

In [18]:
%pip install -qU cassandra-driver python-dotenv

### Iniciar variables de entorno

In [1]:

import os
from dotenv import load_dotenv
from src.python.utils.download_astra_bundle import download_astra_bundle_url


load_dotenv()


ASTRA_DB_DATABASE_ID = os.environ.get("ASTRA_DB_DATABASE_ID")
ASTRA_DB_APPLICATION_TOKEN = os.environ.get("ASTRA_DB_APPLICATION_TOKEN")
ASTRA_DB_KEYSPACE = os.environ.get("ASTRA_DB_KEYSPACE")
SAVE_DIR = '.config/'
SECURE_CONNECT_BUNDLE_PATH = SAVE_DIR + 'secure-connect-bundle.zip'


os.makedirs(SAVE_DIR, exist_ok=True)
download_astra_bundle_url(database_id=ASTRA_DB_DATABASE_ID,
                          token=ASTRA_DB_APPLICATION_TOKEN,
                          out_file_path=SECURE_CONNECT_BUNDLE_PATH)


### Establecer la sesión

In [2]:
#@title Establecer la sesión
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider
from src.python.LexicalSearch import LexicalSearchSession


auth_provider = PlainTextAuthProvider(username="token",
                                      password=os.environ.get("ASTRA_DB_APPLICATION_TOKEN"))

cluster = Cluster(
    cloud={'secure_connect_bundle': SECURE_CONNECT_BUNDLE_PATH},
    auth_provider=auth_provider
)

session = LexicalSearchSession(cluster.connect())


### Probar que podemos conectarnos a la base de datos



In [3]:
from cassandra.query import SimpleStatement
from cassandra.query import dict_factory
from typing import List, Dict
query_stmt = SimpleStatement("SELECT keyspace_name FROM system_schema.keyspaces;")
session.row_factory = dict_factory
rows: List[dict] = session.execute(query_stmt).all()
for row in rows:
    print(row)

Row(keyspace_name='default_keyspace')
Row(keyspace_name='data_endpoint_auth')
Row(keyspace_name='system_auth')
Row(keyspace_name='datastax_sla')
Row(keyspace_name='system_schema')
Row(keyspace_name='cycling')
Row(keyspace_name='system')
Row(keyspace_name='system_traces')


### Crear una tabla

In [7]:
table_name = "cycling_v3"

schema = {
    "record_id": "timeuuid",
    "id": "uuid",
    "commenter": "text",
    "comment": "text",
    "comment_vector": "VECTOR <FLOAT, 5>",
    "created_at": "timestamp",
}

primary_key = ["id", "created_at"]

session.create_table(keyspace=ASTRA_DB_KEYSPACE, 
                     table_name=table_name, 
                     fields=schema, 
                     primary_key=primary_key)


Table cycling.cycling_v3 created successfully.


### Crear índices

Configuraremos dos índices:
1. Para facilitar la búsqueda vectorial
2. Para faciliar la búsqueda léxica. En este caso, utilizaremos el filtro de porterstem, que facilita la coincidencia de word stems usando el algoritmo de Porter stemming. También utilizaremos un filtro de "minúscula" para permitirnos identificar word stems en cualquier caso.

In [8]:
ann_index_name = "ann_index_v3"

session.create_index(keyspace=ASTRA_DB_KEYSPACE,
                     table_name=table_name,
                     field_name="comment_vector",
                     index_name=ann_index_name)

CREATE CUSTOM INDEX ann_index_v3 ON cycling.cycling_v3(comment_vector) USING 'StorageAttachedIndex';


In [10]:
index_analyzer_options = """
    WITH OPTIONS = {
        'index_analyzer': '{
            "tokenizer" : {"name" : "standard"},
            "filters" : [{"name": "lowercase"},{"name" : "porterstem"}]
        }'
    };
"""
lexical_index_name = "lexical_v1"
session.create_index(keyspace=ASTRA_DB_KEYSPACE,
                     table_name=table_name,
                     field_name="comment",
                     index_name=lexical_index_name,
                     index_analyzer_options=index_analyzer_options)

### Adicionar datos


In [11]:
insert_statements = [
    f"""
    INSERT INTO {ASTRA_DB_KEYSPACE}.{table_name} (record_id, id, commenter, comment, created_at, comment_vector)
    VALUES (
        now(),
        e7ae5cf3-d358-4d99-b900-85902fda9bb0,
        'Alex',
        'Raining too hard should have postponed',
        '2017-02-14 12:43:20-0800',
        [0.45, 0.09, 0.01, 0.2, 0.11]
    );""",
    f"""
    INSERT INTO {ASTRA_DB_KEYSPACE}.{table_name} (record_id, id, commenter, comment, created_at, comment_vector)
    VALUES (
        now(),
        e7ae5cf3-d358-4d99-b900-85902fda9bb0,
        'Alex',
        'Second rest stop was out of water',
        '2017-03-21 13:11:09.999-0800',
        [0.99, 0.5, 0.99, 0.1, 0.34]
    );""",
    f"""
    INSERT INTO {ASTRA_DB_KEYSPACE}.{table_name} (record_id, id, commenter, comment, created_at, comment_vector)
    VALUES (
        now(),
        e7ae5cf3-d358-4d99-b900-85902fda9bb0,
        'Alex',
        'LATE RIDERS SHOULD NOT DELAY THE START',
        '2017-04-01 06:33:02.16-0800',
        [0.9, 0.54, 0.12, 0.1, 0.95]
    );""",
    f"""
    INSERT INTO {ASTRA_DB_KEYSPACE}.{table_name} (record_id, id, commenter, comment, created_at, comment_vector)
    VALUES (
        now(),
        c7fceba0-c141-4207-9494-a29f9809de6f,
        'Amy',
        'The gift certificate for winning was the best',
        totimestamp(now()),
        [0.13, 0.8, 0.35, 0.17, 0.03]
    );""",
    f"""
    INSERT INTO {ASTRA_DB_KEYSPACE}.{table_name} (record_id, id, commenter, comment, created_at, comment_vector)
    VALUES (
        now(),
        c7fceba0-c141-4207-9494-a29f9809de6f,
        'Amy',
        'Glad you ran the race in the rain',
        '2017-02-17 12:43:20.234+0400',
        [0.3, 0.34, 0.2, 0.78, 0.25]
    );""",
    f"""
    INSERT INTO {ASTRA_DB_KEYSPACE}.{table_name} (record_id, id, commenter, comment, created_at, comment_vector)
    VALUES (
        now(),
        c7fceba0-c141-4207-9494-a29f9809de6f,
        'Amy',
        'Great snacks at all reststops, but it rained super hard',
        '2017-03-22 5:16:59.001+0400',
        [0.1, 0.4, 0.1, 0.52, 0.09]
    );""",
    f"""
    INSERT INTO {ASTRA_DB_KEYSPACE}.{table_name} (record_id, id, commenter, comment, created_at, comment_vector)
    VALUES (
        now(),
        c7fceba0-c141-4207-9494-a29f9809de6f,
        'Amy',
        'Last climb was a killer',
        '2017-04-01 17:43:08.030+0400',
        [0.3, 0.75, 0.2, 0.2, 0.5]
    );"""
]

# Execute each statement
for stmt in insert_statements:
    session.execute(SimpleStatement(stmt))

### Probar la búsqueda léxica

Emparejaremos "rained", "rain", y "Raining" solo usando el stem "rain".

In [12]:
import json
ann_query = f"""SELECT * FROM {ASTRA_DB_KEYSPACE}.{table_name}
    where comment : 'rain'
    LIMIT 3;"""
rows: List[dict] = session.execute(ann_query).all()
for row in rows:
    print(json.dumps(row, default=str, indent=4))

[
    "e7ae5cf3-d358-4d99-b900-85902fda9bb0",
    "2017-02-14 20:43:20",
    "Raining too hard should have postponed",
    [
        0.44999998807907104,
        0.09000000357627869,
        0.009999999776482582,
        0.20000000298023224,
        0.10999999940395355
    ],
    "Alex",
    "fb1ef370-a12b-11ef-902f-636da99c8be7"
]
[
    "c7fceba0-c141-4207-9494-a29f9809de6f",
    "2017-02-17 08:43:20.234000",
    "Glad you ran the race in the rain",
    [
        0.30000001192092896,
        0.3400000035762787,
        0.20000000298023224,
        0.7799999713897705,
        0.25
    ],
    "Amy",
    "fb3c1860-a12b-11ef-902f-636da99c8be7"
]
[
    "c7fceba0-c141-4207-9494-a29f9809de6f",
    "2017-03-22 01:16:59.001000",
    "Great snacks at all reststops, but it rained super hard",
    [
        0.10000000149011612,
        0.4000000059604645,
        0.10000000149011612,
        0.5199999809265137,
        0.09000000357627869
    ],
    "Amy",
    "fb408530-a12b-11ef-902f-636da99c8be

### Combinar búsqueda léxica con búsqueda vectorial




Podemos filtrar por el stem ("rain" en este caso) y ordenar los resultados por similaridad vectorial.

In [13]:
import json
ann_query = f"""
    SELECT  comment, similarity_cosine(comment_vector, [0.2, 0.15, 0.3, 0.2, 0.05])
      FROM {ASTRA_DB_KEYSPACE}.{table_name}
      WHERE comment: 'snack'
    ORDER BY comment_vector ANN OF [0.1, 0.15, 0.3, 0.12, 0.05]
    LIMIT 3;  """
rows: List[dict] = session.execute(ann_query).all()
for row in rows:
    print(json.dumps(row, default=str, indent=4))

[
    "Great snacks at all reststops, but it rained super hard",
    0.8653713464736938
]


### Más opciones léxicas

Hay más filtros (además de porter stemming). Por favor, vea esta lista: https://docs.datastax.com/en/astra-db-serverless/cql/use-analyzers-with-cql.html#built-in-analyzers